In [2]:
import duckdb as ddb
import pathlib 
import pandas as pd

LOGS_PATH = pathlib.Path(pathlib.Path.cwd()).parent / "data" / "logs_access_logs.json"
OUTPUT_DIR = pathlib.Path(pathlib.Path.cwd()).parent / "data" / "viz"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

## 1. Exploración inicial

In [3]:
con = ddb.connect()

con.execute(f"""
    CREATE TABLE logs AS 
    SELECT * FROM read_json_auto('{LOGS_PATH}')
""")

con.sql("""
        DESCRIBE logs;
        """).show()

┌─────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name     │ column_type │  null   │   key   │ default │  extra  │
│       varchar       │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ log_id              │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ timestamp           │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
│ service_id          │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ server_id           │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ trace_id            │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ span_id             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ method              │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ endpoint            │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ status_code         │ BIGINT      │ YES     │ NULL    │ NULL  

In [4]:
con.sql("""
        SUMMARIZE logs;
        """).show()

┌─────────────────────┬─────────────┬────────────────────────┬────────────────────────┬───────────────┬─────────────────────────┬────────────────────┬────────────────────────────┬────────────────────────────┬────────────────────────────┬───────┬─────────────────┐
│     column_name     │ column_type │          min           │          max           │ approx_unique │           avg           │        std         │            q25             │            q50             │            q75             │ count │ null_percentage │
│       varchar       │   varchar   │        varchar         │        varchar         │     int64     │         varchar         │      varchar       │          varchar           │          varchar           │          varchar           │ int64 │  decimal(9,2)   │
├─────────────────────┼─────────────┼────────────────────────┼────────────────────────┼───────────────┼─────────────────────────┼────────────────────┼────────────────────────────┼────────────────────────────┼

In [5]:
con.sql("""
        SELECT 
            COUNT(*) as total_requests,
            MIN(timestamp) as first_request,
            MAX(timestamp) as last_request,
            COUNT(DISTINCT user_id) as unique_users,
            COUNT(DISTINCT endpoint) as unique_endpoints
        FROM logs;
        """).show()

┌────────────────┬─────────────────────┬─────────────────────┬──────────────┬──────────────────┐
│ total_requests │    first_request    │    last_request     │ unique_users │ unique_endpoints │
│     int64      │      timestamp      │      timestamp      │    int64     │      int64       │
├────────────────┼─────────────────────┼─────────────────────┼──────────────┼──────────────────┤
│           1000 │ 2024-01-03 01:04:13 │ 2025-12-16 13:09:19 │          670 │               11 │
└────────────────┴─────────────────────┴─────────────────────┴──────────────┴──────────────────┘



## 2. Análisis de tráfico

In [43]:
con.sql("""
        SELECT 
            endpoint,
            COUNT(*) as request_count
        FROM logs
        GROUP BY endpoint
        ORDER BY request_count DESC;
        """).show()

┌──────────────────┬───────────────┐
│     endpoint     │ request_count │
│     varchar      │     int64     │
├──────────────────┼───────────────┤
│ /api/products    │           106 │
│ /api/auth/login  │           105 │
│ /api/search      │           104 │
│ /api/auth/logout │            98 │
│ /api/cart        │            98 │
│ /api/users       │            92 │
│ /api/payments    │            89 │
│ /api/orders      │            85 │
│ /api/checkout    │            77 │
│ /health          │            74 │
│ /metrics         │            72 │
├──────────────────┴───────────────┤
│ 11 rows                2 columns │
└──────────────────────────────────┘



## 3. Análisis de errores

In [42]:
con.sql("""
        SELECT 
            status_code,
            COUNT(*) AS status_count
        FROM logs
        GROUP BY status_code
        ORDER BY status_count DESC;
        """).show()

┌─────────────┬──────────────┐
│ status_code │ status_count │
│    int64    │    int64     │
├─────────────┼──────────────┤
│         200 │          286 │
│         201 │           87 │
│         503 │           83 │
│         403 │           76 │
│         502 │           75 │
│         500 │           74 │
│         400 │           72 │
│         401 │           66 │
│         404 │           61 │
│         204 │           60 │
│         301 │           60 │
├─────────────┴──────────────┤
│ 11 rows          2 columns │
└────────────────────────────┘



In [47]:
con.sql("""
        SELECT 
            COUNT(*) AS status_5xx_count
        FROM logs
        WHERE status_code >= 500;
        """).show()

┌──────────────────┐
│ status_5xx_count │
│      int64       │
├──────────────────┤
│              232 │
└──────────────────┘



## 4. Análisis de perfomance 

In [41]:
con.sql("""
        WITH error_5xx AS (
            SELECT 
                endpoint,
                COUNT(*) as error_count
            FROM logs
            WHERE status_code >= 500
            GROUP BY endpoint
        ),
        total_requests AS (
            SELECT 
                endpoint,
                COUNT(*) as total_count
            FROM logs
            GROUP BY endpoint
        ),
        avg_time_response_ms AS (
            SELECT
                endpoint,
                ROUND(AVG(response_time_ms), 2) as avg_response_time_ms
            FROM logs
            GROUP BY endpoint
        )
        SELECT 
            error_5xx.endpoint,
            ROUND(error_5xx.error_count * 100.0 / total_requests.total_count, 2) AS error_percentage,
            error_5xx.error_count,
            total_requests.total_count,
            avg_time_response_ms.avg_response_time_ms
        FROM error_5xx
        JOIN total_requests USING (endpoint)
        JOIN avg_time_response_ms USING (endpoint)
        ORDER BY error_percentage DESC;
        """).show()

┌──────────────────┬──────────────────┬─────────────┬─────────────┬──────────────────────┐
│     endpoint     │ error_percentage │ error_count │ total_count │ avg_response_time_ms │
│     varchar      │      double      │    int64    │    int64    │        double        │
├──────────────────┼──────────────────┼─────────────┼─────────────┼──────────────────────┤
│ /api/cart        │            29.59 │          29 │          98 │              4762.58 │
│ /api/users       │            28.26 │          26 │          92 │              4161.96 │
│ /api/products    │            26.42 │          28 │         106 │              4765.62 │
│ /api/payments    │             23.6 │          21 │          89 │              3658.04 │
│ /api/auth/logout │            23.47 │          23 │          98 │               4048.3 │
│ /metrics         │            22.22 │          16 │          72 │              3679.76 │
│ /api/search      │            22.12 │          23 │         104 │              3377.91 │

In [48]:
con.sql("""
        WITH total_5xx_errors AS (
            SELECT 
                COUNT(*) AS total_5xx_count
            FROM logs
            WHERE status_code >= 500
        ),
        error_5xx AS (
            SELECT 
                endpoint,
                COUNT(*) AS error_count
            FROM logs
            WHERE status_code >= 500
            GROUP BY endpoint
        )
        
        SELECT 
            error_5xx.endpoint,
            error_5xx.error_count,
            total_5xx_errors.total_5xx_count,
            ROUND(error_5xx.error_count * 100.0 / total_5xx_errors.total_5xx_count, 2) AS error_share_percentage
        FROM error_5xx
        CROSS JOIN total_5xx_errors
        ORDER BY error_share_percentage DESC;
        """).show()

┌──────────────────┬─────────────┬─────────────────┬────────────────────────┐
│     endpoint     │ error_count │ total_5xx_count │ error_share_percentage │
│     varchar      │    int64    │      int64      │         double         │
├──────────────────┼─────────────┼─────────────────┼────────────────────────┤
│ /api/cart        │          29 │             232 │                   12.5 │
│ /api/products    │          28 │             232 │                  12.07 │
│ /api/users       │          26 │             232 │                  11.21 │
│ /api/auth/logout │          23 │             232 │                   9.91 │
│ /api/search      │          23 │             232 │                   9.91 │
│ /api/payments    │          21 │             232 │                   9.05 │
│ /api/auth/login  │          21 │             232 │                   9.05 │
│ /api/orders      │          18 │             232 │                   7.76 │
│ /metrics         │          16 │             232 │            

## 5. Análisis temporal

In [49]:
con.sql("""
    WITH daily_stats AS (
        SELECT 
            DATE(timestamp) as date,
            COUNT(*) as requests,
            ROUND(AVG(response_time_ms), 2) as avg_response_time
        FROM logs
        GROUP BY DATE(timestamp)
    )
    SELECT 
        date,
        requests,
        LAG(requests) OVER (ORDER BY date) as previous_day_requests,
        requests - LAG(requests) OVER (ORDER BY date) as difference,
        ROUND(
            (requests - LAG(requests) OVER (ORDER BY date)) * 100.0 / 
            LAG(requests) OVER (ORDER BY date), 
            2
        ) as percent_change
    FROM daily_stats
    ORDER BY date;
        """).show()

┌────────────┬──────────┬───────────────────────┬────────────┬────────────────┐
│    date    │ requests │ previous_day_requests │ difference │ percent_change │
│    date    │  int64   │         int64         │   int64    │     double     │
├────────────┼──────────┼───────────────────────┼────────────┼────────────────┤
│ 2024-01-03 │        1 │                  NULL │       NULL │           NULL │
│ 2024-01-04 │        1 │                     1 │          0 │            0.0 │
│ 2024-01-05 │        1 │                     1 │          0 │            0.0 │
│ 2024-01-06 │        2 │                     1 │          1 │          100.0 │
│ 2024-01-07 │        3 │                     2 │          1 │           50.0 │
│ 2024-01-08 │        1 │                     3 │         -2 │         -66.67 │
│ 2024-01-09 │        1 │                     1 │          0 │            0.0 │
│ 2024-01-11 │        3 │                     1 │          2 │          200.0 │
│ 2024-01-12 │        3 │               

In [50]:
con.sql("""
        WITH daily_stats AS (
            SELECT 
                DATE(timestamp) as date,
                COUNT(*) as requests,
                ROUND(AVG(response_time_ms), 2) as avg_response_time
            FROM logs
            GROUP BY DATE(timestamp)
        ), 
        daily_percent_change AS (
        SELECT 
            date,
            requests,
            requests - LAG(requests) OVER (ORDER BY date) as difference,
            ROUND(
                (requests - LAG(requests) OVER (ORDER BY date)) * 100.0 / 
                LAG(requests) OVER (ORDER BY date), 
                2
            ) as percent_change
        FROM daily_stats
        )
        
        SELECT AVG(percent_change) AS avg_daily_percent_change FROM daily_percent_change;
        """).show()

┌──────────────────────────┐
│ avg_daily_percent_change │
│          double          │
├──────────────────────────┤
│       24.035681818181818 │
└──────────────────────────┘



In [51]:
con.sql("""
        WITH requests_per_hour AS (
            SELECT 
                EXTRACT(HOUR FROM timestamp) AS hour,
                COUNT(*) AS request_count
            FROM logs
            GROUP BY EXTRACT(HOUR FROM timestamp)
            ORDER BY request_count DESC   
        )
        SELECT 
            EXTRACT(HOUR FROM timestamp) AS peak_hour,
            COUNT(*) AS request_count
        FROM logs
        GROUP BY EXTRACT(HOUR FROM timestamp)
        HAVING COUNT(*) > (SELECT AVG(request_count) FROM requests_per_hour)
        ORDER BY request_count DESC
        ;
        """).show()

┌───────────┬───────────────┐
│ peak_hour │ request_count │
│   int64   │     int64     │
├───────────┼───────────────┤
│        17 │            58 │
│         1 │            49 │
│         5 │            49 │
│         4 │            48 │
│        12 │            48 │
│         2 │            46 │
│         7 │            46 │
│        23 │            45 │
│         8 │            43 │
│         9 │            42 │
│        19 │            42 │
├───────────┴───────────────┤
│ 11 rows         2 columns │
└───────────────────────────┘



## 6. Análisis de errores según método HTTP

In [52]:
con.sql("""
        
        WITH errors_by_method AS (
            SELECT 
                method,
                endpoint,
                COUNT(*) AS error_5xx_count,
                ROUND(AVG(response_time_ms), 2) AS avg_response_time_ms
            FROM logs
            WHERE status_code >= 500
            GROUP BY method, endpoint
            ORDER BY error_5xx_count DESC
        ), 
        ranked_errors AS (
            SELECT 
                method, 
                endpoint, 
                error_5xx_count,
                avg_response_time_ms,
                DENSE_RANK() OVER (PARTITION BY method ORDER BY error_5xx_count DESC) AS error_rank
            FROM errors_by_method
        )
        
        SELECT * FROM ranked_errors 
        WHERE error_rank <= 3 AND error_5xx_count > 3;
        """).show()

┌─────────┬──────────────────┬─────────────────┬──────────────────────┬────────────┐
│ method  │     endpoint     │ error_5xx_count │ avg_response_time_ms │ error_rank │
│ varchar │     varchar      │      int64      │        double        │   int64    │
├─────────┼──────────────────┼─────────────────┼──────────────────────┼────────────┤
│ PUT     │ /api/payments    │               5 │              14709.8 │          1 │
│ PUT     │ /api/orders      │               5 │              17903.8 │          1 │
│ PUT     │ /api/auth/login  │               4 │              15770.5 │          2 │
│ PUT     │ /api/products    │               4 │             22521.25 │          2 │
│ PUT     │ /api/users       │               4 │              13657.0 │          2 │
│ DELETE  │ /api/products    │               5 │              15739.4 │          1 │
│ DELETE  │ /api/search      │               4 │              18268.5 │          2 │
│ DELETE  │ /api/auth/logout │               4 │              167

## 7. Exportación para generación de dashboard

In [6]:
# Exportar: Tabla completa de logs con campos derivados para Looker
df_logs = con.sql("""
    SELECT 
        log_id,
        CAST(DATE(timestamp) AS VARCHAR) as date,
        EXTRACT(HOUR FROM timestamp) AS hour,
        EXTRACT(DAYOFWEEK FROM timestamp) AS day_of_week,
        method,
        endpoint,
        status_code,
        CASE 
            WHEN status_code >= 500 THEN 1 
            ELSE 0 
        END AS is_error_5xx,
        CASE 
            WHEN status_code >= 200 AND status_code < 300 THEN 'Success'
            WHEN status_code >= 400 AND status_code < 500 THEN 'Client Error'
            WHEN status_code >= 500 THEN 'Server Error'
            ELSE 'Other'
        END AS status_category,
        response_time_ms,
        user_id,
        user_agent,
        client_ip
    FROM logs
    ORDER BY date, hour;
""").df()

df_logs.to_excel(OUTPUT_DIR / "server_logs_complete.xlsx", index=False)